In [1]:
import json
import requests
from urllib.parse import urlparse, urlunparse

API = "http://wp8m3he1wt.s3-website-ap-southeast-2.amazonaws.com/api/products/1"
CATEGORY = "Air Conditioners"

def get_cubic_weight(width, length, height):
    """Calculate cubic weight

    Cubic weight is calculated by multiplying the length, height and width 
    of the parcel. The result is then multiplied by the industry standard 
    cubic weight conversion factor of 250.

    Arguments
    =========
    width: float
        width of the parcel in centimetres.
    length: float
        length of the parcel in centimetres.
    height: float
        height of the parcel in centimetres.

    Returns
    =======
    cubic_weight: float
        Cubic weight of the parcel in grams.
    """
    assert type(width) ==  float, 'width is not a float'
    assert type(length) ==  float, 'length is not a float'
    assert type(height) ==  float, 'height is not a float'
    assert width > 0, 'width is not positive'
    assert length > 0, 'length is not positive'
    assert height > 0, 'height is not positive'
    
    return width * length * height * 1e-6 * 250 * 1e+3

def get_average_cubic_weight(api, category):
    """Calculate the average cubic weight for all products in the specific category using provided api.

    Get all products data from the paginated api.
    Average cubic weight is calculated by summing up the cubic weight for all 
    products in the specific category then dividing by the amount.

    Arguments
    =========
    api: str
        api endpoint.
    category: str
        category of the product.

    Returns
    =======
    average_cubic_weight: float
        average cubic weight for all products in the specific category.
    """
    assert type(api) ==  str, 'width is not a str'
    assert type(category) ==  str, 'length is not a str'
    
    parse_result = urlparse(api)
    scheme = parse_result.scheme
    netloc = parse_result.netloc
    path = parse_result.path
    cubic_weight = 0
    count = 0
    try:
        while (path is not None):
            url = urlunparse((scheme, netloc, path, "", "", ""))  
            r = requests.get(url, timeout=1)
            data=r.json()
            for product in data["objects"]:
                if product["category"] == category:
                    width = product["size"]["width"]
                    length = product["size"]["length"]
                    height = product["size"]["height"]
                    cubic_weight += get_cubic_weight(width, length, height)
                    count += 1
            path = data["next"]
        if count==0:
            return 0
        else:
            return cubic_weight/count
    except:
        print("Invalid URL: " + url)

print("The average cubic weight for all products in the 'Air Conditioners' category is " + str(get_average_cubic_weight(API, CATEGORY)) + " grams.")

The average cubic weight for all products in the 'Air Conditioners' category is 41613.3846875 grams.
